In [1]:
import sys
sys.path.append("models/")
from setup import out_dir, data_dir, image_dir, model_dir
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
import json
from collections import OrderedDict
import logging
import argparse
import numpy as np
import random
import matplotlib.pyplot as plt
import glob
import pandas as pd

import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torch.backends.cudnn
import torchvision.utils
import torchvision.transforms
import torch.nn.functional as F

try:
    from tensorboardX import SummaryWriter
    is_tensorboard_available = True
except Exception:
    is_tensorboard_available = False

from dataloader import get_loader, image_loader, load_demo
from autoencoder import Autoencoder
from M1_util_train_test import load_model, train, test, AverageMeter
from util_model import my_loss

torch.backends.cudnn.benchmark = True

logging.basicConfig(
    format='[%(asctime)s %(name)s %(levelname)s] - %(message)s',
    datefmt='%Y/%m/%d %H:%M:%S',
    level=logging.DEBUG)
logger = logging.getLogger(__name__)

global_step = 0

[2022/02/02 00:50:57 matplotlib.backends DEBUG] - backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
zoomlevel = 'zoom13'
output_dim = 1
model_run_date = "22013101"
sampling='stratified'
normalization = 'minmax'
variable_names = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
         'pctwhite_alone','pct_nonwhite','pctblack_alone',
         'pct_col_grad','avg_tt_to_work','inc_per_capita']
model_save_variable_names = ['totpop','pct25-34','pct35-50','pctsenior',
         'pctwhite_alone','pct_nonwhite','pctblack_alone',
         'pctcolgrad','avg_tt_to_work','inc']
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

In [3]:
demo_cs, demo_np = load_demo(data_dir, norm=normalization)

[2022/02/01 16:01:53 numexpr.utils INFO] - Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2022/02/01 16:01:53 numexpr.utils INFO] - NumExpr defaulting to 8 threads.


In [4]:
args = {'image_size': 224, 
        'depth': -1,
       'base_channels':64,
       'output_dim':output_dim,
       'num_demo_vars':len(variable_names),
       'demo_norm': normalization,
       'cardinality':1,
       'epochs':200,
       'batch_size':16,
       'base_lr':0.005,
       'weight_decay':0.0005,
       'momentum': 0.9,
       'nesterov': True,
       'milestones': '[50,100]',
       'lr_decay':0.1,
       'seed': 1234,
       'outdir':out_dir,
       'num_workers':8,
       'tensorboard':False,
       'save':True}

model_config = OrderedDict([
    ('arch', 'resnext'),
    ('depth', args['depth']),
    ('base_channels', args['base_channels']),
    ('cardinality', args['cardinality']),
    ('input_shape', (1, 3, 32, 32)),
    ('output_dim', args['output_dim']),
    ('num_demo_vars', args['num_demo_vars'])
])

optim_config = OrderedDict([
    ('epochs', args['epochs']),
    ('batch_size', args['batch_size']),
    ('base_lr', args['base_lr']),
    ('weight_decay', args['weight_decay']),
    ('momentum', args['momentum']),
    ('nesterov', args['nesterov']),
    ('milestones', json.loads(args['milestones'])),
    ('lr_decay', args['lr_decay']),
])

data_config = OrderedDict([
    ('dataset', 'CIFAR10'),
    ('image_size', args['image_size']),
    ('demo_norm', args['demo_norm'])
])

run_config = OrderedDict([
    ('seed', args['seed']),
    ('outdir', args['outdir']),
    ('save', args['save']),
    ('num_workers', args['num_workers']),
    ('tensorboard', args['tensorboard']),
])

config = OrderedDict([
    ('model_config', model_config),
    ('optim_config', optim_config),
    ('data_config', data_config),
    ('run_config', run_config),
])

In [5]:
# parse command line arguments
#config = parse_args()
#logger.info(json.dumps(config, indent=2))

model_name = datetime.now().strftime("%m%d-%H%M")

run_config = config['run_config']
optim_config = config['optim_config']

# TensorBoard SummaryWriter
writer = SummaryWriter(model_name) if run_config['tensorboard'] else None

# set random seed
seed = run_config['seed']
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# create output directory
outdir = run_config['outdir']
if not os.path.exists(outdir):
    os.makedirs(outdir)

# save config as json file in output directory
outpath = os.path.join(outdir, 'config.json')
with open(outpath, 'w') as fout:
    json.dump(config, fout, indent=2)

In [6]:
# data loaders
# train_loader, test_loader = get_loader(optim_config['batch_size'], run_config['num_workers'])
train_loader, test_loader = image_loader(image_dir+zoomlevel+"/", data_dir, optim_config['batch_size'], 
                                         run_config['num_workers'], 
                                         data_config['image_size'], sampling=sampling, 
                                         recalculate_normalize=False)

In [7]:
#criterion = nn.MSELoss(reduction='mean')
criterion = my_loss

In [8]:
# model
config['model_config']['input_shape'] = (1,3,data_config['image_size'],data_config['image_size'])

encoder = load_model(config['model_config']['arch'], 'Encoder', config['model_config'])
encoder = encoder.to(device)

config['model_config']['input_shape'] = [1,2048,config['model_config']['output_dim'],config['model_config']['output_dim']]

config['model_config']['conv_shape'] = [data_config['image_size']//32,data_config['image_size']//32]
config['model_config']['output_channels'] = 3

decoder = load_model(config['model_config']['arch'], 'Decoder', config['model_config'])
decoder = decoder.to(device)

config['encoder'] = encoder
config['decoder'] = decoder
model = load_model('autoencoder','Autoencoder', config)
model = model.to(device)

n_params = sum([param.view(-1).size()[0] for param in encoder.parameters()]) +\
           sum([param.view(-1).size()[0] for param in decoder.parameters()])
logger.info('n_params: {}'.format(n_params))

# optimizer
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=optim_config['base_lr'],
    momentum=optim_config['momentum'],
    weight_decay=optim_config['weight_decay'],
    nesterov=optim_config['nesterov'])

scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer,
    milestones=optim_config['milestones'],
    gamma=optim_config['lr_decay'])

# Test with Adam Optimizer (easier to setup, experiment with SGD later)
# optimizer = torch.optim.Adam(model.parameters(), lr=optim_config['base_lr'],
#                              weight_decay=optim_config['weight_decay'])

# run test before start training
# test_outputs = test(0, model, criterion, test_loader, run_config, writer, device)

ref1 = 0
ref2 = 0

for epoch in range(optim_config['epochs']):

    loss_ = train(epoch, model, optimizer, criterion, train_loader, (demo_cs,demo_np), run_config,
         writer, device, logger=logger)

    scheduler.step()

    test(epoch, model, criterion, test_loader, (demo_cs,demo_np), run_config,
                    writer, device, logger, return_output=False)

    if epoch % 5 == 0:
        if epoch > 50:
            if (np.abs(loss_ - ref1)/ref1<ref1*0.01) & (np.abs(loss_ - ref2)/ref2<ref2*0.01):
                print("Early stopping at epoch", epoch)
                break
            if (ref1 < loss_) & (ref1 < ref2):
                print("Diverging. stop.")
                break
            if loss_ < best:
                best = loss_
                best_epoch = epoch
        else:
            best = loss_
            best_epoch = epoch

        ref2 = ref1
        ref1 = loss_

        if (config['run_config']['save']) & (best_epoch==epoch):
            torch.save({'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()},
                model_dir+"SAE_"+zoomlevel+"_"+str(model_config['output_dim']**2*2048)+"_"+
                model_run_date+"_"+str(epoch)+".pt")

            
if config['run_config']['save']:
    files = glob.glob(model_dir+"SAE_"+zoomlevel+"_"+str(model_config['output_dim']**2*2048)+"_"+
                              model_run_date+"_*.pt")

    for f in files:
        e = int(f.split("_")[-1].split(".")[0])
        if e != best_epoch:
            os.remove(f)

        
if run_config['tensorboard']:
    outpath = os.path.join(outdir, 'all_scalars.json')
    writer.export_scalars_to_json(outpath)
    

[2022/02/01 16:02:43 __main__ INFO] - n_params: 31961862
[2022/02/01 16:04:08 __main__ INFO] - Epoch 0 Step 603/604 Train Loss 0.85085648
[2022/02/01 16:04:13 __main__ INFO] - Epoch 0 Test Loss 0.75235411
[2022/02/01 16:04:13 __main__ INFO] - Elapsed 4.45
[2022/02/01 16:05:38 __main__ INFO] - Epoch 1 Step 603/604 Train Loss 0.76819638
[2022/02/01 16:05:42 __main__ INFO] - Epoch 1 Test Loss 0.71209569
[2022/02/01 16:05:42 __main__ INFO] - Elapsed 4.12
[2022/02/01 16:07:08 __main__ INFO] - Epoch 2 Step 603/604 Train Loss 0.74185187
[2022/02/01 16:07:12 __main__ INFO] - Epoch 2 Test Loss 0.69355645
[2022/02/01 16:07:12 __main__ INFO] - Elapsed 4.09
[2022/02/01 16:08:37 __main__ INFO] - Epoch 3 Step 603/604 Train Loss 0.72086907
[2022/02/01 16:08:41 __main__ INFO] - Epoch 3 Test Loss 0.68010988
[2022/02/01 16:08:41 __main__ INFO] - Elapsed 4.13
[2022/02/01 16:10:07 __main__ INFO] - Epoch 4 Step 603/604 Train Loss 0.71043354
[2022/02/01 16:10:11 __main__ INFO] - Epoch 4 Test Loss 0.66393446

[2022/02/01 17:03:55 __main__ INFO] - Elapsed 4.13
[2022/02/01 17:05:21 __main__ INFO] - Epoch 41 Step 603/604 Train Loss 0.45228433
[2022/02/01 17:05:25 __main__ INFO] - Epoch 41 Test Loss 0.54598538
[2022/02/01 17:05:25 __main__ INFO] - Elapsed 4.09
[2022/02/01 17:06:51 __main__ INFO] - Epoch 42 Step 603/604 Train Loss 0.44457460
[2022/02/01 17:06:55 __main__ INFO] - Epoch 42 Test Loss 0.55876570
[2022/02/01 17:06:55 __main__ INFO] - Elapsed 4.09
[2022/02/01 17:08:20 __main__ INFO] - Epoch 43 Step 603/604 Train Loss 0.44322673
[2022/02/01 17:08:24 __main__ INFO] - Epoch 43 Test Loss 0.54128374
[2022/02/01 17:08:24 __main__ INFO] - Elapsed 4.12
[2022/02/01 17:09:50 __main__ INFO] - Epoch 44 Step 603/604 Train Loss 0.43888614
[2022/02/01 17:09:54 __main__ INFO] - Epoch 44 Test Loss 0.55531756
[2022/02/01 17:09:54 __main__ INFO] - Elapsed 4.09
[2022/02/01 17:11:19 __main__ INFO] - Epoch 45 Step 603/604 Train Loss 0.43842999
[2022/02/01 17:11:23 __main__ INFO] - Epoch 45 Test Loss 0.5341

[2022/02/01 18:05:09 __main__ INFO] - Elapsed 4.13
[2022/02/01 18:06:35 __main__ INFO] - Epoch 82 Step 603/604 Train Loss 0.36675891
[2022/02/01 18:06:39 __main__ INFO] - Epoch 82 Test Loss 0.50107226
[2022/02/01 18:06:39 __main__ INFO] - Elapsed 4.10
[2022/02/01 18:08:04 __main__ INFO] - Epoch 83 Step 603/604 Train Loss 0.36521679
[2022/02/01 18:08:08 __main__ INFO] - Epoch 83 Test Loss 0.51445395
[2022/02/01 18:08:08 __main__ INFO] - Elapsed 4.07
[2022/02/01 18:09:34 __main__ INFO] - Epoch 84 Step 603/604 Train Loss 0.36300899
[2022/02/01 18:09:38 __main__ INFO] - Epoch 84 Test Loss 0.50478182
[2022/02/01 18:09:38 __main__ INFO] - Elapsed 4.10
[2022/02/01 18:11:03 __main__ INFO] - Epoch 85 Step 603/604 Train Loss 0.36105962
[2022/02/01 18:11:07 __main__ INFO] - Epoch 85 Test Loss 0.49882293
[2022/02/01 18:11:07 __main__ INFO] - Elapsed 4.13
[2022/02/01 18:12:33 __main__ INFO] - Epoch 86 Step 603/604 Train Loss 0.36135683
[2022/02/01 18:12:37 __main__ INFO] - Epoch 86 Test Loss 0.4981

[2022/02/01 19:06:23 __main__ INFO] - Epoch 122 Test Loss 0.48479881
[2022/02/01 19:06:23 __main__ INFO] - Elapsed 4.07
[2022/02/01 19:07:48 __main__ INFO] - Epoch 123 Step 603/604 Train Loss 0.31728988
[2022/02/01 19:07:52 __main__ INFO] - Epoch 123 Test Loss 0.50357324
[2022/02/01 19:07:52 __main__ INFO] - Elapsed 4.08
[2022/02/01 19:09:18 __main__ INFO] - Epoch 124 Step 603/604 Train Loss 0.31880784
[2022/02/01 19:09:22 __main__ INFO] - Epoch 124 Test Loss 0.47958803
[2022/02/01 19:09:22 __main__ INFO] - Elapsed 4.12
[2022/02/01 19:10:47 __main__ INFO] - Epoch 125 Step 603/604 Train Loss 0.31661012
[2022/02/01 19:10:52 __main__ INFO] - Epoch 125 Test Loss 0.47441961
[2022/02/01 19:10:52 __main__ INFO] - Elapsed 4.10
[2022/02/01 19:12:17 __main__ INFO] - Epoch 126 Step 603/604 Train Loss 0.31552783
[2022/02/01 19:12:21 __main__ INFO] - Epoch 126 Test Loss 0.47763879
[2022/02/01 19:12:21 __main__ INFO] - Elapsed 4.13
[2022/02/01 19:13:47 __main__ INFO] - Epoch 127 Step 603/604 Train L

Diverging. stop.


In [ ]:
loss_meter_1 = AverageMeter()
loss_meter_2 = AverageMeter()

for step, (image_list, data) in enumerate(test_loader):

    census_index = [demo_cs.index(i[i.rfind('/')+1:i.rfind('_')]) for i in image_list]
    census_data = demo_np[census_index]

    census_data = torch.tensor(census_data).to(device)
    data = data.to(device)

    out_image, out_demo = model(data)

    loss1, loss2 = criterion(out_image, out_demo, data, census_data, return_components=True)

    num = data.size(0)

    loss_meter_1.update(loss1.item(), num)
    loss_meter_2.update(loss2.item(), num)

    if step % 10 == 0:
        print(step, end='\t')

best_test_1 = loss_meter_1.avg
best_test_2 = loss_meter_2.avg
print(best_test_1, best_test_2)         

loss_meter_1 = AverageMeter()
loss_meter_2 = AverageMeter()                                                              
for step, (image_list, data) in enumerate(train_loader):

    census_index = [demo_cs.index(i[i.rfind('/')+1:i.rfind('_')]) for i in image_list]
    census_data = demo_np[census_index]

    census_data = torch.tensor(census_data).to(device)
    data = data.to(device)

    out_image, out_demo = model(data)

    loss1, loss2 = criterion(out_image, out_demo, data, census_data, return_components=True)

    num = data.size(0)

    loss_meter_1.update(loss1.item(), num)
    loss_meter_2.update(loss2.item(), num)

    if step % 10 == 0:
        print(step, end='\t')

best_1 = loss_meter_1.avg
best_2 = loss_meter_2.avg
print(best_1, best_2)

In [9]:
with open(out_dir+"SAE_train.csv", "a") as f:
    f.write("%s,%s,%d,%s,%s,%d,%.4f,%.4f,%.4f,%.4f\n" % (model_run_date, zoomlevel, model_config['output_dim']**2*2048, 
            sampling, normalization, best_epoch, best_1, best_2, best_test_1, best_test_2))

0	10	20	30	40	50	60	70	0.29800634201653975 0.014136222738354649
0	10	20	30	40	50	60	70	80	90	100	110	120	130	140	150	160	170	180	190	200	210	220	230	240	250	260	270	280	290	300	310	320	330	340	350	360	370	380	390	400	410	420	430	440	450	460	470	480	490	500	510	520	530	540	550	560	570	580	590	600	0.27115582356488466 0.001241747316502718


In [ ]:
# Reverse Normalization

# CIFAR
# inv_normalize = torchvision.transforms.Normalize(
#     mean=[-0.4914/0.2470, -0.4822/0.2435, -0.4465/0.2616],
#     std=[1/0.2470, 1/0.2435, 1/0.2616]
# )


# Satellite image
inv_normalize = torchvision.transforms.Normalize(
    mean=[-0.3733/0.2173, -0.3991/0.2055, -0.3711/0.2143],
    std=[1/0.2173, 1/0.2055, 1/0.2143]
)

In [ ]:
for step, (_,data) in enumerate(test_loader):
    data = data.to(device)
    test_output = model(data)
    test_output_orig = inv_normalize(test_output)
    data_orig = inv_normalize(data)
    if step == 0:
        break

In [ ]:
plot_image = 15

In [ ]:
plt.hist(data[plot_image, :, :, :].cpu().detach().numpy().flatten())

In [ ]:
plt.hist(test_output[plot_image, :, :, :].cpu().detach().numpy().flatten())

In [ ]:
torch.mean(test_output[:,0,:,:])

In [ ]:
torch.mean(test_output[:,1,:,:])

In [ ]:
torch.mean(test_output[:,2,:,:])

In [ ]:
torch.mean(data[:,0,:,:])

In [ ]:
torch.mean(data[:,1,:,:])

In [ ]:
torch.mean(data[:,1,:,:])

In [ ]:
np.mean((test_output - data).detach().cpu().numpy()[:,0,:,:])

In [ ]:
np.mean((test_output - data).detach().cpu().numpy()[:,1,:,:])

In [ ]:
np.mean((test_output - data).detach().cpu().numpy()[:,2,:,:])

In [ ]:
np.mean((test_output - data).cpu().detach().numpy())

In [ ]:
np.mean(np.power((test_output - data).cpu().detach().numpy(), 2))

In [ ]:
plt.hist((test_output - data).detach().cpu().numpy()[:,2,:,:].flatten())

In [ ]:
plt.hist((test_output - data).detach().cpu().numpy()[:,1,:,:].flatten());

In [ ]:
plt.hist((test_output - data).detach().cpu().numpy()[:,0,:,:].flatten())

In [ ]:
test_output_orig[plot_image,:,:,:].cpu().detach().permute(1, 2, 0).numpy()

In [ ]:
data_orig[plot_image,:,:,:].cpu().detach().permute(1, 2, 0).numpy()

In [ ]:
plot_image = 3

In [ ]:
plt.imshow(test_output_orig[plot_image,:,:,:].cpu().detach().permute(1, 2, 0))
plt.axis('off');

In [ ]:
plt.imshow(data_orig[plot_image,:,:,:].cpu().detach().permute(1, 2, 0));
plt.axis('off');

In [ ]:
model

In [ ]:
plt.hist(data.detach().numpy()[:,0,:,:].flatten())